In [2]:
# Import dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data Preparation

In [3]:
# Generate population DataFrame from csv file
raw_pop_data = pd.read_csv("../country_data/pop.csv", encoding='unicode_escape')

# Remove unnecessary columns
raw_pop_data = raw_pop_data[["Country (or dependency)", "Population"]]

# Remove invalid entries (NaN)
pop_data = raw_pop_data.dropna()


# Rename columns
pop_data.columns = ["Country", "Population"]

In [4]:
# Generate age DataFrame from csv file
raw_age_data = pd.read_csv("../country_data/age.csv")

# Take the columns we want
raw_age_data = raw_age_data[["country", "MedianAge2023"]]

# Remove invalid entries (NaN)
age_data = raw_age_data.dropna()

# Rename columns
raw_age_data.columns = ["Country", "Median Age"]

In [5]:
# Generate GDP DataFrame from csv file
raw_gdp_data = pd.read_csv("../country_data/gdp.csv")
raw_gdp_data

# Take the columns we want
raw_gdp_data = raw_gdp_data[["country", "GdpPerCapitaIMF_USD_23"]]

# Remove invalid entries (NaN)
gdp_data = raw_gdp_data.dropna()

# Rename columns
gdp_data.columns = ["Country", "GDP Per Capita (USD)"]

In [6]:
# Generate Facebook DataFrame from csv file
raw_fb_data = pd.read_csv("platform_data/facebook.csv")

# Take columns we want
fb_data = raw_fb_data[["country", "FacebookUsersTotal2023"]]

# Rename columns
fb_data.columns = ["Country", "Facebook Users"]

In [7]:
# Generate TikTok DataFrame from csv file
tt_data = pd.read_csv("platform_data/tiktok.csv")

# Rename columns
tt_data.columns = ["Country", "TikTok Users"]


In [11]:
# Generate YouTube DataFrame from csv file
raw_yt_data = pd.read_csv("platform_data/youtube.csv")

# Take columns we want
yt_data = raw_yt_data[["country", "YouTubeUsersJuly2023"]]

# Rename columns
yt_data.columns = ["Country", "YouTube Users"]

# Since users is measured in units of millions, we need to convert
yt_data["YouTube Users"] = yt_data['YouTube Users']*1000000

C:\Users\Moburu\AppData\Local\Temp\ipykernel_28292\1115339961.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yt_data["YouTube Users"] = yt_data['YouTube Users']*1000000


,Country,YouTube Users
0,India,467000000.0
1,United States,246000000.0
2,Indonesia,139000000.0
3,Pakistan,71700000.0
4,Nigeria,31600000.0
...,...,...
82,Slovenia,1640000.0
83,Latvia,1480000.0
84,Bahrain,1460000.0
85,Estonia,1070000.0
